In [7]:
!pip install openpyxl --break-system-packages

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [12]:
import os
import torch
import plotly.graph_objects as go
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from torch.utils.data import DataLoader
from openpyxl import Workbook
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math
import torch
import torch.nn as nn
import math

def smape(y_true, y_pred):
    return 100 * torch.mean(2 * torch.abs(y_true - y_pred) / (torch.abs(y_true) + torch.abs(y_pred))).item()

def rmse(y_true, y_pred):
    return torch.sqrt(torch.mean((y_true - y_pred) ** 2))
    
def mae(y_true, y_pred):
    return torch.mean(torch.abs(y_true - y_pred))

def minmax_scale(tensor):
    tensor_min = tensor.min()
    tensor_max = tensor.max()
    scaled_tensor = (tensor - tensor_min) / (tensor_max - tensor_min)
    return scaled_tensor, tensor_min, tensor_max

def inverse_minmax_scale(scaled_tensor, tensor_min, tensor_max):
    original_tensor = scaled_tensor * (tensor_max - tensor_min) + tensor_min
    return original_tensor

window_size = 24
horizon = 24
train_prop = 0.6
val_prop = 0.2

df = pd.read_csv('datasets/usage/house1.csv')

class SlidingWindowsDataset(torch.utils.data.Dataset):
    def __init__(self, data, window_size=24, horizon=24):
        self.data = data
        self.window_size = window_size
        self.horizon = horizon
        
        self.feature_mins = self.data.min(axis=0).values
        self.feature_maxs = self.data.max(axis=0).values
    
    def __len__(self):
        return len(self.data) - (self.window_size + self.horizon) + 1

    def __getitem__(self, idx):
        window = self.data[idx: idx + self.window_size + self.horizon]
        past_all_features = window[:self.window_size]
        future_energy = window[self.window_size:, -1:]
        
        scaled_past_all_features = (past_all_features - self.feature_mins) / (self.feature_maxs - self.feature_mins)
        
        scaled_future_energy, energy_min, energy_max = minmax_scale(future_energy)

        return scaled_past_all_features, scaled_future_energy, (energy_min, energy_max)

df_dict = {
    'house2': df,
}

train_datasets = {}
val_datasets = {}
test_datasets = {}

for dataset_name, dataset_df in df_dict.items():
    data_tensor = torch.from_numpy(dataset_df['PJME_MW'].values).float()
    data_shape = data_tensor.shape[0]
    train_end = int(data_shape * train_prop)
    val_end = int(data_shape * (train_prop + val_prop))
    train_data = data_tensor[:train_end]
    val_data = data_tensor[train_end:val_end]
    test_data = data_tensor[val_end:]
    train_datasets[dataset_name] = SlidingWindowsDataset(train_data, window_size, horizon)
    val_datasets[dataset_name] = SlidingWindowsDataset(val_data, window_size, horizon)
    test_datasets[dataset_name] = SlidingWindowsDataset(test_data, window_size, horizon)

def investigate_nan_in_dataframe(df, dataset_name):
    if df.isnull().any().any():
        print(f"NaN values detected in dataset: {dataset_name}")
        
        nan_columns = df.columns[df.isnull().any()].tolist()
        print(f"Columns with NaN values: {nan_columns}")
        
        for col in nan_columns:
            nan_rows = df.index[df[col].isnull()].tolist()
            print(f"Rows with NaN values in column {col}: {nan_rows}")
    else:
        print(f"No NaN values detected in dataset: {dataset_name}")

for dataset_name, dataset_df in df_dict.items():
    investigate_nan_in_dataframe(dataset_df, dataset_name)

def print_sample_data(dataset_name, dataset, sample_index=0):
    past_features, future_energy, energy_min_max = dataset[sample_index]
    
    inversed_future_energy = inverse_minmax_scale(future_energy, *energy_min_max)
    
    print(f"Dataset Name: {dataset_name}")
    print("Past 24 hours' features:")
    print(past_features)
    print("\nNext 24 hours' scaled energy values:")
    print(future_energy)
    print("\nInverse scaled energy values for next 24 hours:")
    print(inversed_future_energy)
    print("\nMin-Max values used for scaling:")
    print(energy_min_max)
    print("-----------------------------------------------------------")
for dataset_name in train_datasets:
    print_sample_data(dataset_name + " Train", train_datasets[dataset_name])

for dataset_name in val_datasets:
    print_sample_data(dataset_name + " Validation", val_datasets[dataset_name])

for dataset_name in test_datasets:
    print_sample_data(dataset_name + " Test", test_datasets[dataset_name])

No NaN values detected in dataset: house2


IndexError: too many indices for tensor of dimension 1

In [13]:
# Transormer "Attention All you Need"
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(1, 0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        return x + self.pe[:x.size(0), :]

#Transformer without Decoder 
class TransformerWOD(nn.Module):
    def __init__(self, num_features, d_model, nhead, num_layers):
        super(TransformerWOD, self).__init__()
        self.embedding = nn.Linear(num_features, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers)
        self.fc = nn.Linear(d_model, 1)
    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer.encoder(x)
        output = self.fc(x)
        return output

#Transformer with Decoder 
class TransformerWD(nn.Module):
    def __init__(self, num_features, d_model, nhead, num_layers, num_decoder_layers=None):
        super(TransformerWD, self).__init__()
        self.embedding = nn.Linear(num_features, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        if num_decoder_layers is None:
            num_decoder_layers = num_layers  
        self.transformer = nn.Transformer(d_model, nhead, num_layers, num_decoder_layers)
        self.fc = nn.Linear(d_model, 1)  

    def forward(self, src, tgt=None, src_mask=None, tgt_mask=None, src_key_padding_mask=None, tgt_key_padding_mask=None):
        src = self.embedding(src)
        src = self.pos_encoder(src)
        
        if tgt is None:
            tgt = src  
        else:
            tgt = self.embedding(tgt)
            tgt = self.pos_encoder(tgt)
        memory = self.transformer.encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
        output = self.transformer.decoder(tgt, memory, tgt_mask=tgt_mask, memory_mask=None, 
                                           tgt_key_padding_mask=tgt_key_padding_mask, memory_key_padding_mask=src_key_padding_mask)
        output = self.fc(output)
        return output

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print("Training will be performed on GPU")
else:
    print("Training will be performed on CPU")
learning_rate = 0.001
def train(model, data_loader, optimizer, loss_fn):
    model.train()
    losses = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}

    for inputs, scaled_targets, future_energy_min_max in data_loader:
        inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
        targets_min, targets_max = future_energy_min_max
        targets_min, targets_max = targets_min.to(device), targets_max.to(device)
        
        outputs = model(inputs)
        loss = loss_fn(outputs, scaled_targets)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

        targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
        #print("Inverse Scaled Targets:", targets[0].cpu().numpy())  # Print the first inverse scaled target
        outputs = inverse_minmax_scale(outputs, targets_min, targets_max)
        #print("Inverse Scaled Outputs:", outputs[0].detach().cpu().numpy())    # Print the first inverse scaled output

        metrics['MAE'].append(mae(outputs, targets).item())
        metrics['RMSE'].append(rmse(outputs, targets).item())
        metrics['SMAPE'].append(smape(outputs, targets))

    return np.mean(losses), {k: np.mean(v) for k, v in metrics.items()}

def evaluate(model, data_loader, loss_fn):
    model.eval()
    losses = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}

    with torch.no_grad():
        for inputs, scaled_targets, future_energy_min_max in data_loader:
            inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
            targets_min, targets_max = future_energy_min_max
            targets_min, targets_max = targets_min.to(device), targets_max.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, scaled_targets)
            losses.append(loss.item())

            targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
            outputs = inverse_minmax_scale(outputs, targets_min, targets_max)

            metrics['MAE'].append(mae(outputs, targets).item())
            metrics['RMSE'].append(rmse(outputs, targets).item())
            metrics['SMAPE'].append(smape(outputs, targets))

    return np.mean(losses), {k: np.mean(v) for k, v in metrics.items()}
    

model = TransformerWOD(num_features=6, d_model=128, nhead=4, num_layers=2).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-7)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)
loss_fn = nn.L1Loss()

def train_and_validate(datasets, num_epochs, patience,loss_fn):
    plots_dir = "./Transformer_MSE/"
    
    for dataset_name in datasets['train'].keys():
        print(f"Training for dataset: {dataset_name}")
        
        train_loader = DataLoader(datasets['train'][dataset_name], batch_size=1, shuffle=False)
        val_loader = DataLoader(datasets['val'][dataset_name], batch_size=1, shuffle=False)
        
        best_val_loss = np.inf
        epochs_no_improve = 0

        train_loss_history = []
        val_loss_history = []
        train_mae_history = []
        val_mae_history = []
        train_rmse_history = []
        val_rmse_history = []
        train_smape_history = []
        val_smape_history = []

        for epoch in range(num_epochs):
            train_loss, train_metrics = train(model, train_loader, optimizer, loss_fn)
            val_loss, val_metrics = evaluate(model, val_loader, loss_fn)
            #scheduler.step(val_loss)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

            if epochs_no_improve == patience:
                print(f"Early stopping triggered after {epoch + 1} epochs for dataset {dataset_name}.")
                break

            train_loss_history.append(train_loss)
            val_loss_history.append(val_loss)
            train_mae_history.append(train_metrics['MAE'])
            val_mae_history.append(val_metrics['MAE'])
            train_rmse_history.append(train_metrics['RMSE'])
            val_rmse_history.append(val_metrics['RMSE'])
            train_smape_history.append(train_metrics['SMAPE'])
            val_smape_history.append(val_metrics['SMAPE'])

            print(f"Epoch {epoch + 1}/{num_epochs} for dataset {dataset_name}")
            print(f"Train Loss: {train_loss:.2f}, Validation Loss: {val_loss:.2f}")
            print(f"Train MAE: {train_metrics['MAE']:.2f}, Validation MAE: {val_metrics['MAE']:.2f}")
            print(f"Train RMSE: {train_metrics['RMSE']:.2f}, Validation RMSE: {val_metrics['RMSE']:.2f}")
            print(f"Train SMAPE: {train_metrics['SMAPE']:.2f}, Validation SMAPE: {val_metrics['SMAPE']:.2f}")

        # Create plots and save them for each dataset
        def plot_metrics(history1, history2, title, ylabel, filename):
            fig = go.Figure()
            fig.add_trace(go.Scatter(y=history1, mode='lines', name='Train'))
            fig.add_trace(go.Scatter(y=history2, mode='lines', name='Validation'))
            fig.update_layout(title=title, xaxis_title='Epoch', yaxis_title=ylabel)
            fig.write_image(f"{plots_dir}/{dataset_name}_{filename}")

        plot_metrics(train_loss_history, val_loss_history, 'Loss vs Epoch', 'Loss', 'drop_loss_plot.png')
        plot_metrics(train_mae_history, val_mae_history, 'MAE vs Epoch', 'MAE', 'drop_mae_plot.png')
        plot_metrics(train_rmse_history, val_rmse_history, 'RMSE vs Epoch', 'RMSE', 'drop_rmse_plot.png')
        plot_metrics(train_smape_history, val_smape_history, 'SMAPE vs Epoch', 'SMAPE', 'drop_smape_plot.png')

        # Save the model for each dataset
        torch.save(model.state_dict(), f'./Transformer_MSE/Transformer_{loss_fn}_{dataset_name}.pth')

datasets = {
    'train': train_datasets,
    'val': val_datasets,
    'test': test_datasets
}



patience = 7
num_epochs = 100

train_and_validate(datasets, num_epochs, patience,loss_fn)

Training will be performed on CPU


/opt/homebrew/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Training for dataset: house2


/opt/homebrew/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


IndexError: too many indices for tensor of dimension 1

In [ ]:
def test(model, data_loader, loss_fn):
    model.eval()
    losses = []
    all_outputs = []
    all_targets = []
    metrics = {'MAE': [], 'RMSE': [], 'SMAPE': []}

    with torch.no_grad():
        for inputs, scaled_targets, future_energy_min_max in data_loader:
            inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
            targets_min, targets_max = future_energy_min_max
            targets_min, targets_max = targets_min.to(device), targets_max.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, scaled_targets)
            losses.append(loss.item())

            targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max)
            outputs = inverse_minmax_scale(outputs, targets_min, targets_max)

            metrics['MAE'].append(mae(outputs, targets).item())
            metrics['RMSE'].append(rmse(outputs, targets).item())
            metrics['SMAPE'].append(smape(outputs, targets))

    return np.mean(losses), {k: np.mean(v) for k, v in metrics.items()}

# Load saved models and test
test_results = {}
for dataset_name in datasets['test'].keys():
    print(f"Testing for dataset: {dataset_name}")
    model.load_state_dict(torch.load(f'./Transformer_MSE/Transformer_{loss_fn}_{dataset_name}.pth'))
    test_loader = DataLoader(datasets['test'][dataset_name], batch_size=1, shuffle=False)

    test_loss, test_metrics = test(model, test_loader, loss_fn)
    
    print(f"Test Loss for {dataset_name}: {test_loss:.2f}")
    print(f"Test MAE for {dataset_name}: {test_metrics['MAE']:.2f}")
    print(f"Test RMSE for {dataset_name}: {test_metrics['RMSE']:.2f}")
    print(f"Test SMAPE for {dataset_name}: {test_metrics['SMAPE']:.2f}")
    print('-' * 50)

    test_results[dataset_name] = {
        'loss': test_loss,
        'MAE': test_metrics['MAE'],
        'RMSE': test_metrics['RMSE'],
        'SMAPE': test_metrics['SMAPE']
    }

def test_with_values(model, data_loader, loss_fn):
    model.eval()
    all_outputs = []
    all_targets = []

    with torch.no_grad():
        for inputs, scaled_targets, future_energy_min_max in data_loader:
            inputs, scaled_targets = inputs.to(device), scaled_targets.to(device)
            targets_min, targets_max = future_energy_min_max
            targets_min, targets_max = targets_min.to(device), targets_max.to(device)

            outputs = model(inputs)

            targets = inverse_minmax_scale(scaled_targets, targets_min, targets_max).cpu().numpy()
            outputs = inverse_minmax_scale(outputs, targets_min, targets_max).cpu().numpy()

            all_outputs.extend(outputs)
            all_targets.extend(targets)
        
    return all_targets, all_outputs

# Load saved models, test and save to CSV
for dataset_name in datasets['test'].keys():
    print(f"Generating values for dataset: {dataset_name}")
    model.load_state_dict(torch.load(f'./Transformer_MSE/Transformer_{loss_fn}_{dataset_name}.pth'))

    test_loader = DataLoader(datasets['test'][dataset_name], batch_size=1, shuffle=False)

    actual_values, predicted_values = test_with_values(model, test_loader, loss_fn)
    print(len(np.ravel(actual_values)))
    print(len(np.ravel(predicted_values)))
    
    '''
    df_results = pd.DataFrame({
        'Actual': actual_values,
        'Predicted': predicted_values
    })
    '''
    df_results = pd.DataFrame({
        'Actual': np.ravel(actual_values),
        'Predicted': np.ravel(predicted_values)
    })
    df_results.to_csv(f"./Transformer_MSE/{dataset_name}_results.csv", index=False)
    print(f"Saved results for {dataset_name} to {dataset_name}_results.csv")